**Assignment 7.1 : Classifications and Regression**

Name: Ignacio, Edrie<br>
Course and Section: CPE019 - CPE32S1<br>
Date of Submission: 06/30/2024<br>
Instructor: Engr. Roman M. Richard<br>

1. Choose any dataset applicable to the classification problem, and also, choose any dataset applicable to the regression problem.

Install and import the needed libraries

In [ ]:
!pip install keras

In [ ]:
!pip install scikeras

In [ ]:
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56441 sha256=84e04489cc202da843ef4f5b29912ac66dc5db1aa55e9a4fdab8fe0936ca9919
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import gzip
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder

Set a random seed

In [ ]:
seed = 7
np.random.seed(seed)

Read the dataset

In [ ]:
df = pd.read_csv("ObesityDataSet.csv")

Display the dataset

In [ ]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


Display the data types of the features

In [ ]:
df.dtypes

Gender                             object
Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight     object
FAVC                               object
FCVC                              float64
NCP                               float64
CAEC                               object
SMOKE                              object
CH2O                              float64
SCC                                object
FAF                               float64
TUE                               float64
CALC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object

Drop the features that has a data type of object

In [ ]:
obesedf = df.drop(columns=['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS', 'NObeyesdad'])

print(obesedf)

            Age    Height      Weight  FCVC  NCP      CH2O       FAF       TUE
0     21.000000  1.620000   64.000000   2.0  3.0  2.000000  0.000000  1.000000
1     21.000000  1.520000   56.000000   3.0  3.0  3.000000  3.000000  0.000000
2     23.000000  1.800000   77.000000   2.0  3.0  2.000000  2.000000  1.000000
3     27.000000  1.800000   87.000000   3.0  3.0  2.000000  2.000000  0.000000
4     22.000000  1.780000   89.800000   2.0  1.0  2.000000  0.000000  0.000000
...         ...       ...         ...   ...  ...       ...       ...       ...
2106  20.976842  1.710730  131.408528   3.0  3.0  1.728139  1.676269  0.906247
2107  21.982942  1.748584  133.742943   3.0  3.0  2.005130  1.341390  0.599270
2108  22.524036  1.752206  133.689352   3.0  3.0  2.054193  1.414209  0.646288
2109  24.361936  1.739450  133.346641   3.0  3.0  2.852339  1.139107  0.586035
2110  23.664709  1.738836  133.472641   3.0  3.0  2.863513  1.026452  0.714137

[2111 rows x 8 columns]


Displays the updated dataset

In [ ]:
obesedf

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
0,21.000000,1.620000,64.000000,2.0,3.0,2.000000,0.000000,1.000000
1,21.000000,1.520000,56.000000,3.0,3.0,3.000000,3.000000,0.000000
2,23.000000,1.800000,77.000000,2.0,3.0,2.000000,2.000000,1.000000
3,27.000000,1.800000,87.000000,3.0,3.0,2.000000,2.000000,0.000000
4,22.000000,1.780000,89.800000,2.0,1.0,2.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
2106,20.976842,1.710730,131.408528,3.0,3.0,1.728139,1.676269,0.906247
2107,21.982942,1.748584,133.742943,3.0,3.0,2.005130,1.341390,0.599270
2108,22.524036,1.752206,133.689352,3.0,3.0,2.054193,1.414209,0.646288
2109,24.361936,1.739450,133.346641,3.0,3.0,2.852339,1.139107,0.586035


# 2. Explain your datasets and the problem being addressed.


**Classification(ObesityDataSet):** The original data for the analysis that I selected is the Estimation of Obesity Levels Based On Eating Habits and Physical Condition. The data set in this case is cleaned to only show float data types between ‘Age’ and ‘TUE’. The rationale for selecting this dataset is to identify that area where Obese people require enhancement.

---

**Regression(Wholesale customers data):** In this case, I have chosen such a dataset in order to observe, which Region has most of its purchases in wholesale.

# 3. For classification, do the following:


Pick the features and select a target variable

In [ ]:
obesedf_features = ['Age', 'Height', 'Weight', 'FCVC',
                   'NCP', 'CH2O', 'FAF', 'TUE']

# Defining the content of the target_column
obesedf_target = 'Weight'

Split the values into X and Y

In [ ]:
# Split the dataset into features (X) and the target variable (y)
X = obesedf[obesedf_features].values
Y = obesedf[obesedf_target].values  # Use the target variable as it is

In [ ]:
y_binary = np.where(Y > 5, 1, 0)

Create a base model

In [ ]:
def modified_baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Use appropriate loss function and metrics for regression
    return model

# Define the model using the modified_baseline_model function
estimator = KerasClassifier(build_fn=modified_baseline_model, epochs=20, batch_size=5, verbose=1)

# Define the cross-validation strategy
kfold = KFold(n_splits=10, shuffle=True)

# Perform cross-validation
results = cross_val_score(estimator, X, y_binary, cv=kfold)

# Print the results
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7044 - loss: 2.8881
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.5196
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.4057
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.3185
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2523
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.2018
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1630
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1328
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1089
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0899
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0747
Epoch 12/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9174 - loss: 0.1802
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0023
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 9.3647e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.3675e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.5983e-04
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.6546e-04
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.0768e-04
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 7.9244e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.2150e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.2966e-05
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.1297e-05
Epoch 12/20
380/380 ━━━━━━━

...

380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4244 - loss: 4.5496
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9998 - loss: 0.5294
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.4189
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.3311
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2634
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2113
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1709
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1394
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1144
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.0945
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0785
Epoch 12/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8147 - loss: 0.7989
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0034
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.2298e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.9447e-06
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.8549e-06
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.5092e-06
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.8664e-07
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.3412e-07
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.9975e-07
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.4310e-07
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.5858e-07
Epoch 12/20
380/380 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3781 - loss: 1.0464
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0237
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0039
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0015
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 8.0281e-04
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.4359e-04
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.4210e-04
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.6599e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 3.9511e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 2.3021e-05
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.4973e-05
Epoch 12/20
380/380 ━━━━━━━━━━━━━━━

Modify the model

In [ ]:
def modified_baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Use appropriate loss function and metrics for regression
    return model

# Define the model using the modified_baseline_model function
estimator = KerasClassifier(build_fn=modified_baseline_model, epochs=20, batch_size=5, verbose=1)

# Define the cross-validation strategy
kfold = KFold(n_splits=10, shuffle=True)

# Perform cross-validation
results = cross_val_score(estimator, X, y_binary, cv=kfold)

# Print the results
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7474 - loss: 1.3113
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0010
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 2.6422e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 1.1388e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.3843e-05
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.0005e-05
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.9305e-05
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.2005e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 8.5610e-06
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.3712e-06
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.4739e-06
Epoch 12/20
380/380 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8573 - loss: 0.4356
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0022
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.1948e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.7301e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.5340e-04
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 8.8905e-05
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.0718e-05
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.0127e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.6426e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.0178e-05
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.5350e-05
Epoch 12/20
380/380 ━━━━━━━

...

380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9160 - loss: 0.6699
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.4961
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.3821
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2978
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2350
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.1876
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1513
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1232
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1011
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0835
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0693
Epoch 12/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 1.6499e-10
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.8144e-11
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.7097e-11
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.1538e-11
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.4847e-11
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 8.3680e-12
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 9.2673e-12
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.4404e-12
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.2034e-12
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 6.0810e-12
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.9655e-12
Epoch 12/20
380/380

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2514 - loss: 6.5876
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.5338
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.4287
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.3421
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2737
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.2204
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.1786
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 0.1458
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 0.1198
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0990
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0822
Epoch 12/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy

Improve the accuracy of your model by applying additional hidden layers

In [ ]:
def modified_baseline_model2():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Use appropriate loss function and metrics for regression
    return model

# Define the model using the modified_baseline_model function
estimator = KerasClassifier(build_fn=modified_baseline_model2, epochs=20, batch_size=5, verbose=1)

# Define the cross-validation strategy
kfold = KFold(n_splits=10, shuffle=True)

# Perform cross-validation
results = cross_val_score(estimator, X, y_binary, cv=kfold)

# Print the results
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5797 - loss: 6.5314
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 8.3491e-04
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.0764e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.3793e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 9.0438e-05
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.9146e-05
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.7637e-05
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.7341e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.9499e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.5311e-05
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.2982e-05
Epoch 12/20
380/380 ━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0107
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.1402e-04
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 3.4584e-05
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.6179e-05
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 9.6018e-06
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 6.5062e-06
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.4922e-06
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 2.7835e-06
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.4360e-06
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.8605e-06
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.2665e-06
Epoch 12/20
380/380 ━━━

...

380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5315 - loss: 7.9922
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 2.7767e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.0111e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.4421e-05
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.9829e-05
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.0573e-05
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.5963e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 1.0695e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.3861e-06
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.6672e-06
Epoch 12/20
380/380 ━━━━━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0022
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 5.9850e-07
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.4150e-08
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.0857e-08
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.8559e-09
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.6619e-09
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 2.1045e-09
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.5771e-09
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.2375e-09
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 8.2553e-10
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.1615e-10
Epoch 12/20
380/380 ━━━

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


380/380 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8686 - loss: 0.5654
Epoch 2/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.8052e-04
Epoch 3/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.4078e-04
Epoch 4/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.6869e-04
Epoch 5/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 1.3321e-04
Epoch 6/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.4915e-05
Epoch 7/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 7.4627e-05
Epoch 8/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 4.1512e-05
Epoch 9/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 3.1371e-05
Epoch 10/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 3.0157e-05
Epoch 11/20
380/380 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.0000 - loss: 2.4121e-05
Epoch 12/20
380/380 ━━━

# 4. For regression, do the following:

Upload the dataset for Regression

In [ ]:
wsale = pd.read_csv("Wholesale customers data.csv")

Wholesale datatypes

In [ ]:
wsale.dtypes

Channel             int64
Region              int64
Fresh               int64
Milk                int64
Grocery             int64
Frozen              int64
Detergents_Paper    int64
Delicassen          int64
dtype: object

Display the features of the dataset

In [ ]:
wsale

,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
...,...,...,...,...,...,...,...,...
435,1,3,29703,12051,16027,13135,182,2204
436,1,3,39228,1431,764,4510,93,2346
437,2,3,14531,15488,30243,437,14841,1867
438,1,3,10290,1981,2232,1038,168,2125


Select the feature values and the target value

In [ ]:
#Defining the contents of the forest_feature
wsale_feature =  ['Channel', 'Region', 'Fresh', 'Milk', 'Grocery', 'Frozen',
                   'Detergents_Paper', 'Delicassen']

# Defining the contents of the forest_target
wsale_target = 'Region'

Split the Data in wX and wY

In [ ]:
# Split the dataset into features (X) and the target variable (y)
wX = wsale[wsale_feature].values
wY = wsale[wsale_target].values

Create the base model

In [ ]:
def wsale_baseline_model():
	# create model
  model = Sequential()
  model.add(Dense(10, input_shape=(8,), kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))

  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

Model evaluation

In [ ]:
#evaluate model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wsale_baseline_model, epochs=20, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, wX, wY, cv=kfold, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.5841
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4422
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.6232
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4963
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5440
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2552
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2033
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1361
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1134
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0990
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0976
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0898
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0650
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0563
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0606
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.6568
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.4073
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.1732
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2221
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4339
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2037
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1522
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1263
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0778
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0738
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0627
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0557
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0463
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0382
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0401
Epoch 16/20
80/80 ━━━━━━━━━━━━

...

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.9043
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.5950
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.0831
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0141
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3049
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1981
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1825
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1174
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1042
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0841
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0838
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0675
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0568
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0491
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0459
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.6446
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.8282
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.6040
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.5067
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6073
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2949
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2415
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1678
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1588
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0969
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0969
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0815
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0808
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0509
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0445
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 6.9594
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.4885
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.1952
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1609
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3399
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1213
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0957
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0668
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0830
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0628
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0483
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0572
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0436
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0500
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0348
Epoch 16/20
80/80 ━━━━━━━━━━━━

Larger Network

In [ ]:
def wsale_baseline_model2():
	# create model
  model = Sequential()
  model.add(Dense(12, input_shape=(8,), kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(Dense(1, kernel_initializer='normal'))

  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

#evaluate model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(model=wsale_baseline_model2, epochs=20, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, wX, wY, cv=kfold, scoring='neg_mean_squared_error')
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.6681
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3684
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.7778
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2712
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1612
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1156
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0834
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0507
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0555
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0387
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0335
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0259
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0238
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0172
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0159
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.7513
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3879
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.8044
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2366
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1383
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0975
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0833
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0686
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0454
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0407
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0363
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0265
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0215
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0197
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0140
Epoch 16/20
80/80 ━━━━━━━━━━━━

...

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7.2161
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 5.6193
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.7951
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1909
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0781
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0503
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0435
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0362
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0305
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0320
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0213
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0191
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0168
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0150
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0109
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.5677
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8436
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7362
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1833
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1235
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0994
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0633
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0597
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0438
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0363
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0276
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0257
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0189
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0154
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0147
Epoch 16/20
80/80 ━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 6.5901
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.3132
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4248
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2608
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1342
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1005
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0793
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0619
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0481
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0499
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0371
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0317
Epoch 13/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0248
Epoch 14/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0213
Epoch 15/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0206
Epoch 16/20
80/80 ━━━━━━━━━━━━

Smaller Network

In [ ]:
from sklearn.model_selection import StratifiedKFold
def create_smaller():
	# create model
  model = Sequential()
  model.add(Dense(5, input_shape=(8,), activation='relu'))
  model.add(Dense(1))
	# Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_smaller, epochs=20, batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, wX, wY, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0835 - loss: 8.2941
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1206 - loss: 3.2228
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1053 - loss: 1.5344
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1120 - loss: -0.3407
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1199 - loss: -2.1172
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1561 - loss: -2.4877
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1299 - loss: -2.6135
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1206 - loss: -3.3048
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1387 - loss: -2.5025
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1625 - loss: -3.1389
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1503 - loss: -3.0311
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1676 - loss: 15.1500
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1786 - loss: 10.3612
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1852 - loss: 7.8528
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1641 - loss: 2.4689
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1655 - loss: 3.3230
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1801 - loss: 1.1606
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2147 - loss: 0.8412
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1852 - loss: -0.4890
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2003 - loss: -1.4296
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2243 - loss: -0.4531
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1736 - loss: -2.2629
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1812 - loss: -

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1705 - loss: 23.4242
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2053 - loss: 22.5580
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1695 - loss: 23.3854
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1396 - loss: 23.9114
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1712 - loss: 22.7919
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1947 - loss: 22.4623
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1631 - loss: 22.6474
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1745 - loss: 22.7857
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1757 - loss: 20.9978
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1673 - loss: 20.6076
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1582 - loss: 18.4195
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1763 - lo

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1932 - loss: 18.3262
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1819 - loss: 15.9038
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2057 - loss: 12.2185
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1442 - loss: 8.6255
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2043 - loss: 2.3593
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1614 - loss: 0.6229
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1790 - loss: 0.3772
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1619 - loss: -0.3481
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1746 - loss: -0.8844
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1698 - loss: -1.7107
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1417 - loss: -1.1331
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1782 - loss: 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0592 - loss: 4.7357
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0520 - loss: 1.5732
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0976 - loss: 0.0680
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1076 - loss: -2.5564
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.1051 - loss: -2.6055
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0925 - loss: -3.8640
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1280 - loss: -4.5359
Epoch 8/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1070 - loss: -6.1115
Epoch 9/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1023 - loss: -7.2885
Epoch 10/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0802 - loss: -7.8345
Epoch 11/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1279 - loss: -6.9723
Epoch 12/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0958 - loss

# Summary and Lesson Learned

Summary:

- This activity helped me understand more about Classifications and Regressions. I used the Data set Obesity Data Set for Classification and Wholesale customers data for Regression. In the regression part, the output of my standard deviation is -0.06 (0.08) MSE, -0.02 (0.03) MSE in the Larger Netork, and Smaller: 17.50% (1.04%) in the Small Network.

Lessons Learned:

- I have understood the concept of layer and neurons in a model, how to upgrade a model using the standard deviation of the dataset, and the two approaches of regression and classification in constructing as well as assessing a base model; the last lesson is understanding how to assess a model using K-fold cross validation.